# Year 2023 Day 16

[--- Day 16: The Floor Will Be Lava ---](https://adventofcode.com/2023/day/16)


## Motive

A video was produced before this notebook, available here: [video problem 202316](/videos/2023/video_problem_202316.html)

This notebook takes a step back and try to visualize with simpler tools, while reusing some code from the video generation (notably the history generation).

## Related puzzles

- [--- Day 14: Parabolic Reflector Dish ---](https://adventofcode.com/2023/day/14)


In [1]:
from typing import Any, Callable

import numpy as np
import pandas as pd
import xarray as xr
from pyobsplot import Plot

In [2]:
from advent_of_code.y_2023.problem_202316 import AdventOfCodeProblem202316

problem = AdventOfCodeProblem202316()
problem

AdventOfCodeProblem202316(year=2023, day=16)

In [3]:
board = xr.DataArray(
    data=problem.parse_input_text_file(),
    dims=("row", "col"),
)
board

<xarray.DataArray (row: 112, col: 112)>
array([[ 79,  79,  79, ...,  79,  79,  79],
       [ 79,  92,  46, ..., 124,  46,  79],
       [ 79,  46,  45, ...,  46,  46,  79],
       ...,
       [ 79,  46, 124, ...,  46,  46,  79],
       [ 79,  46,  46, ...,  46,  46,  79],
       [ 79,  79,  79, ...,  79,  79,  79]], dtype=uint8)
Dimensions without coordinates: row, col

In [4]:
from advent_of_code.visualization.observable_plot import ObservablePlotXarrayBuilder

## With Example Input


In [5]:
from advent_of_code.common.common import get_example_inputs_file_contents

example_input = get_example_inputs_file_contents(2023)["test_problem_202316"][
    "EXAMPLE_INPUT"
]
parsed_example_input = xr.DataArray(
    problem.parse_text_input(example_input), dims=("row", "col")
)

In [6]:
from advent_of_code.visualization.observable_plot import ascii_2d_numpy_array_plotter


example_input_plotter = ascii_2d_numpy_array_plotter(
    parsed_example_input,
    obfuscate=False,
    text=True,
    width=300,
    height=300,
)

In [7]:
example_input_plotter.plot()

Layer #0 <function ObservablePlotXarrayBuilder.create_raster_background_marks.<locals>.callback at 0x7fa37e4a1620>
Layer #1 <function text_mark_producer.<locals>.callback at 0x7fa383b696c0>


We can see the 4 types of mirrors: `-`, `|`, `/` and `\`. Note that a new symbol, `O`, has been introduced. It represented walls. It is a help to manage incoming rays that are outside of the board, and avoid out of bounds coordinates.

Let's get fancy and replace the ASCII characters by Unicode characters, representing more faithfully the mirrors.

See

- [\ on Wikipédia](https://en.wiktionary.org/wiki/%5C) and [U+FF3C: FULLWIDTH REVERSE SOLIDUS](https://charbase.com/ff3c-unicode-fullwidth-reverse-solidus) (`＼`)
- [/ on Wikipédia](https://en.wiktionary.org/wiki//) and [U+FF0F: FULLWIDTH SOLIDUS](https://charbase.com/ff0f) (`／`)
- [Box-drawing characters](https://en.wikipedia.org/wiki/Box-drawing_characters) (`─` and `│`)


In [8]:
parsed_example_input

<xarray.DataArray (row: 12, col: 12)>
array([[ 79,  79,  79,  79,  79,  79,  79,  79,  79,  79,  79,  79],
       [ 79,  46, 124,  46,  46,  46,  92,  46,  46,  46,  46,  79],
       [ 79, 124,  46,  45,  46,  92,  46,  46,  46,  46,  46,  79],
       [ 79,  46,  46,  46,  46,  46, 124,  45,  46,  46,  46,  79],
       [ 79,  46,  46,  46,  46,  46,  46,  46,  46, 124,  46,  79],
       [ 79,  46,  46,  46,  46,  46,  46,  46,  46,  46,  46,  79],
       [ 79,  46,  46,  46,  46,  46,  46,  46,  46,  46,  92,  79],
       [ 79,  46,  46,  46,  46,  47,  46,  92,  92,  46,  46,  79],
       [ 79,  46,  45,  46,  45,  47,  46,  46, 124,  46,  46,  79],
       [ 79,  46, 124,  46,  46,  46,  46,  45, 124,  46,  92,  79],
       [ 79,  46,  46,  47,  47,  46, 124,  46,  46,  46,  46,  79],
       [ 79,  79,  79,  79,  79,  79,  79,  79,  79,  79,  79,  79]],
      dtype=uint8)
Dimensions without coordinates: row, col

In [9]:
from IPython.display import display, Markdown


def to_unicode_board(puzzle_xda: xr.DataArray):
    mapping = {
        "/": "\uff3c",
        "\\": "\uff0f",
        # "-": "\u2500",
        "-": "―",  # best rendered in plot https://www.compart.com/en/unicode/U+2015
        "|": "\u2502",
        # "|": "—",
        ".": "·",
        # "O": "⯀",
        "O": "□",
    }
    unicode_board = puzzle_xda.copy(data=puzzle_xda.values.view("S1").astype("str"))

    # unicode_board[:, 0] = "┃"
    # unicode_board[:, -1] = "┃"
    # unicode_board[0, :] = "━"
    # unicode_board[-1, :] = "━"
    # unicode_board[0, -1] = "┓"
    # unicode_board[-1, 0] = "┗"
    # unicode_board[-1, -1] = "┛"
    # unicode_board[0, 0] = "┏"

    for old, new in mapping.items():
        unicode_board = unicode_board.where(unicode_board != old, new)
    return unicode_board


unicode_board = to_unicode_board(parsed_example_input)
rendered = "\n".join("".join(row) for row in unicode_board.values.tolist())
display(Markdown(f"```\n{rendered}\n```"))

```
□□□□□□□□□□□□
□·│···／····□
□│·―·／·····□
□·····│―···□
□········│·□
□··········□
□·········／□
□····＼·／／··□
□·―·―＼··│··□
□·│····―│·／□
□··＼＼·│····□
□□□□□□□□□□□□
```

In [10]:
unicode_board.dtype

dtype('<U1')

In the following, the walls are removed for cleaner visualization.

In [11]:
unicode_plotter = ascii_2d_numpy_array_plotter(
    unicode_board[1:-1, 1:-1],
    obfuscate=False,
    text=True,
    width=300,
    height=300,
    scheme="Category10",
)
unicode_plotter.plot()

Layer #0 <function ObservablePlotXarrayBuilder.create_raster_background_marks.<locals>.callback at 0x7fa37e4df880>
Layer #1 <function text_mark_producer.<locals>.callback at 0x7fa37e4df920>


In [12]:
(unicode_plotter.copy(label=False)
 .unstack(0)
 ).plot()

Layer #0 <function text_mark_producer.<locals>.callback at 0x7fa37e4df920>


#### Compute the history

In [13]:
history_raw = problem.compute_history(parsed_example_input, 250, verbose=False)
history_raw

[(0, [1, 0], [0, 1]),
 (1, [1, 1], [0, 1]),
 (2, [1, 2], [1, 0]),
 (2, [1, 2], [-1, 0]),
 (3, [2, 2], [1, 0]),
 (4, [3, 2], [1, 0]),
 (5, [4, 2], [1, 0]),
 (6, [5, 2], [1, 0]),
 (7, [6, 2], [1, 0]),
 (8, [7, 2], [1, 0]),
 (9, [8, 2], [0, 1]),
 (9, [8, 2], [0, -1]),
 (10, [8, 3], [0, 1]),
 (11, [8, 4], [0, 1]),
 (12, [8, 5], [-1, 0]),
 (13, [7, 5], [0, 1]),
 (14, [7, 6], [0, 1]),
 (15, [7, 7], [1, 0]),
 (16, [8, 7], [1, 0]),
 (17, [9, 7], [0, 1]),
 (17, [9, 7], [0, -1]),
 (18, [9, 8], [1, 0]),
 (18, [9, 8], [-1, 0]),
 (18, [9, 6], [0, -1]),
 (19, [10, 8], [1, 0]),
 (19, [8, 8], [-1, 0]),
 (19, [9, 5], [0, -1]),
 (20, [7, 8], [0, -1]),
 (20, [9, 4], [0, -1]),
 (21, [7, 7], [-1, 0]),
 (21, [9, 3], [0, -1]),
 (22, [6, 7], [-1, 0]),
 (22, [9, 2], [1, 0]),
 (22, [9, 2], [-1, 0]),
 (23, [5, 7], [-1, 0]),
 (23, [8, 2], [0, 1]),
 (23, [8, 2], [0, -1]),
 (23, [10, 2], [1, 0]),
 (24, [4, 7], [-1, 0]),
 (24, [8, 1], [0, -1]),
 (25, [3, 7], [0, 1]),
 (25, [3, 7], [0, -1]),
 (26, [3, 8], [0, 1]),
 (

In [14]:
len(history_raw)

61

In [15]:
history = [
    (depth, tuple(position), tuple(speed)) for (depth, position, speed) in history_raw
]
history

[(0, (1, 0), (0, 1)),
 (1, (1, 1), (0, 1)),
 (2, (1, 2), (1, 0)),
 (2, (1, 2), (-1, 0)),
 (3, (2, 2), (1, 0)),
 (4, (3, 2), (1, 0)),
 (5, (4, 2), (1, 0)),
 (6, (5, 2), (1, 0)),
 (7, (6, 2), (1, 0)),
 (8, (7, 2), (1, 0)),
 (9, (8, 2), (0, 1)),
 (9, (8, 2), (0, -1)),
 (10, (8, 3), (0, 1)),
 (11, (8, 4), (0, 1)),
 (12, (8, 5), (-1, 0)),
 (13, (7, 5), (0, 1)),
 (14, (7, 6), (0, 1)),
 (15, (7, 7), (1, 0)),
 (16, (8, 7), (1, 0)),
 (17, (9, 7), (0, 1)),
 (17, (9, 7), (0, -1)),
 (18, (9, 8), (1, 0)),
 (18, (9, 8), (-1, 0)),
 (18, (9, 6), (0, -1)),
 (19, (10, 8), (1, 0)),
 (19, (8, 8), (-1, 0)),
 (19, (9, 5), (0, -1)),
 (20, (7, 8), (0, -1)),
 (20, (9, 4), (0, -1)),
 (21, (7, 7), (-1, 0)),
 (21, (9, 3), (0, -1)),
 (22, (6, 7), (-1, 0)),
 (22, (9, 2), (1, 0)),
 (22, (9, 2), (-1, 0)),
 (23, (5, 7), (-1, 0)),
 (23, (8, 2), (0, 1)),
 (23, (8, 2), (0, -1)),
 (23, (10, 2), (1, 0)),
 (24, (4, 7), (-1, 0)),
 (24, (8, 1), (0, -1)),
 (25, (3, 7), (0, 1)),
 (25, (3, 7), (0, -1)),
 (26, (3, 8), (0, 1)),
 (

In [16]:
def create_ray_plot_df(history: list[tuple[int, list[int], list[int]]]) -> pd.DataFrame:
    future_df = []
    for depth, pos, speed in history:
        if speed[0] == 0:
            char = "―"
        else:
            char = "\u2502"
        new_entry = (pos[0], pos[1], depth, char)
        future_df.append(new_entry)

    ray_df = pd.DataFrame(future_df, columns=["x", "y", "depth", "char"])
    return ray_df


ray_df = create_ray_plot_df(history)
ray_df

,x,y,depth,char
0,1,0,0,―
1,1,1,1,―
2,1,2,2,│
3,1,2,2,│
4,2,2,3,│
...,...,...,...,...
56,1,3,31,―
57,9,6,32,│
58,1,2,32,│
59,1,2,32,│


In [17]:
def plot_rays(
    board: xr.DataArray,
    ray_df: pd.DataFrame,
    *,
    width: int = 300,
    dx: int = 13,
    dy: int = 13,
    scheme: str = "turbo",
    laser: bool = False,
    font_coefficient: int = 40,
    strokeWidth: int = 2,
):
    plotter = ascii_2d_numpy_array_plotter(
        # board[1:-1, 1:-1],
        board,
        obfuscate=False,
        text=True,
        width=width,
        height=width,
        label=False,
        color={"legend": True, "scheme": scheme, "label": "Ray's depth"},
    ).unstack(0)

    x_target = "y"
    y_target = "x"
    plotter.insert(
        lambda: [
            Plot.text(  # type:ignore
                ray_df,
                {
                    x_target: "x",
                    y_target: "y",
                    "text": "char",
                    "stroke": "depth",
                    "strokeWidth": strokeWidth,
                    "fill": "currentColor" if laser else None,
                    "dy": dx,
                    "dx": dy,
                    "fontSize": font_coefficient * width / 600 * 13 / board["col"].size,
                },
            )
        ],
        0,
    )
    return plotter


plot_rays(unicode_board, ray_df, width=300).plot()

Layer #0 <function plot_rays.<locals>.<lambda> at 0x7fa37e469940>
Layer #1 <function text_mark_producer.<locals>.callback at 0x7fa37e22c180>


## With Actual Input


In [18]:
actual_unicode_board = to_unicode_board(
    xr.DataArray(problem.parse_input_text_file(), dims=("row", "col"))
)
actual_unicode_board

<xarray.DataArray (row: 112, col: 112)>
array([['□', '□', '□', ..., '□', '□', '□'],
       ['□', '／', '·', ..., '│', '·', '□'],
       ['□', '·', '―', ..., '·', '·', '□'],
       ...,
       ['□', '·', '│', ..., '·', '·', '□'],
       ['□', '·', '·', ..., '·', '·', '□'],
       ['□', '□', '□', ..., '□', '□', '□']], dtype='<U1')
Dimensions without coordinates: row, col

In [19]:
plot_input = False

In [20]:
if plot_input:
    unicode_plotter = (
        ascii_2d_numpy_array_plotter(
            actual_unicode_board[1:-1, 1:-1],
            obfuscate=False,
            text=True,
            width=840,
            height=840,
            scheme="Category10",
        )
    )
    unicode_plotter.plot()
    
    unicode_plotter.copy(
        label=False,
        width=840,
        height=840,
    ).unstack(0).plot()


Load History


In [21]:
import json
from pathlib import Path


actual_history_raw = json.loads(
    (
        Path().resolve()
        / "../../../.."
        / problem.get_visualizations_instructions_for_part_1_file_path()
    )
    .resolve()
    .read_text()
)

In [22]:
len(actual_history_raw)

10356

Because the history is so long, we must transform it from "raster" to "vector". More precisely, the rays explored the board cell by cell, but not this exploration is done, we can regroup a sequence of consecutive moves in the same direction into a single line, that can be plotted with Observable Plot.

In [23]:
actual_history = [
    (depth, tuple(position), tuple(speed)) for (depth, position, speed) in actual_history_raw
]
actual_history[:5]

[(0, (1, 0), (0, 1)),
 (1, (1, 1), (1, 0)),
 (2, (2, 1), (1, 0)),
 (3, (3, 1), (1, 0)),
 (4, (4, 1), (1, 0))]

In [24]:
actual_ray_df = create_ray_plot_df(actual_history)

In [25]:
plot_rays(
    actual_unicode_board,
    actual_ray_df,
    width=900,
    dx=4,
    dy=4,
    strokeWidth=2, # font_coefficient=10,
).plot()

Layer #0 <function plot_rays.<locals>.<lambda> at 0x7fa37e4a18a0>
Layer #1 <function text_mark_producer.<locals>.callback at 0x7fa383b691c0>


## Improvements Ideas

The rays do not chain together perfectly well. It is because only their position is considered, and not their direction. Using the direction would help offsetting the unicode characters in a better way, so that the rays really "reflects" on the mirror, with an exact junction. However, this might require more coding, as currently, only the position is taken into account, and this ray's history is flat. Using the full exploration tree might help ; even better, instrumenting the problem's code to produce visualization input material at the same time. Same discussions as described in the notebook 202317 apply here (_during execution_ versus _after execution_ visualization code production). A good example is [Notebok Problem 202318](./notebook_problem_202318.ipynb)